In [1]:
import os

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from pandas.tseries.frequencies import to_offset

In [2]:
print(*os.listdir("./data"), sep="\n")

wrangled_data
orig_data


In [3]:
orig_data_dir = "./data/orig_data/"
print(*os.listdir(orig_data_dir), sep="\n")

PricesFile1.csv
NG_ImpliedVols2020.csv
NG_ImpliedVols2022.csv
NG_ImpliedVols2021.csv
WTI_ImpliedVols2022.csv
WTI_ImpliedVols2020.csv
WTI_ImpliedVols2021.csv
InterestRates.csv


In [4]:
prices_df = pd.read_csv(orig_data_dir+"PricesFile1.csv")
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130 entries, 0 to 6129
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   priceindex  6130 non-null   object 
 1   pricedate   6130 non-null   object 
 2   delivdate   6130 non-null   object 
 3   price       6130 non-null   float64
dtypes: float64(1), object(3)
memory usage: 191.7+ KB


In [5]:
prices_df.head()

,priceindex,pricedate,delivdate,price
0,AECO NIT MONTHLY (7A) US$,2020-03-01,2020-03-01,1.3219
1,AECO NIT MONTHLY (7A) US$,2020-04-01,2020-04-01,1.2421
2,AECO NIT MONTHLY (7A) US$,2020-05-01,2020-05-01,1.3385
3,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-04-01,-0.5140
4,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.6160


In [6]:
prices_df.pricedate, prices_df.delivdate = pd.to_datetime(prices_df.pricedate), pd.to_datetime(prices_df.delivdate)
prices_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6130 entries, 0 to 6129
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   priceindex  6130 non-null   object        
 1   pricedate   6130 non-null   datetime64[ns]
 2   delivdate   6130 non-null   datetime64[ns]
 3   price       6130 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 191.7+ KB


In [7]:
prices_df.rename(columns={"priceindex" : "index", "pricedate" : "t", "delivdate" : "T", "price" : "F"}, 
                 inplace=True)
prices_df.head()

,index,t,T,F
0,AECO NIT MONTHLY (7A) US$,2020-03-01,2020-03-01,1.3219
1,AECO NIT MONTHLY (7A) US$,2020-04-01,2020-04-01,1.2421
2,AECO NIT MONTHLY (7A) US$,2020-05-01,2020-05-01,1.3385
3,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-04-01,-0.5140
4,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.6160


In [8]:
p_i = prices_df["index"].unique()
print(*p_i, sep="\n")

AECO NIT MONTHLY (7A) US$
AECO NIT MONTHLY (7A) US$ FWD
NYMEX Natural Gas
WTI NYMEX LIGHT SWEET


In [9]:
prices_df["index"].value_counts()

WTI NYMEX LIGHT SWEET            2060
AECO NIT MONTHLY (7A) US$ FWD    2049
NYMEX Natural Gas                2018
AECO NIT MONTHLY (7A) US$           3
Name: index, dtype: int64

In [10]:
grouped_by_p_i = prices_df.groupby("index")

In [11]:
for i in range(len(p_i)):
    
    df = grouped_by_p_i.get_group(p_i[i])
    print("index : " + p_i[i])
    print("t_min is " + str(min(df.t).date()) + "\t" + "t_max is " + str(max(df.t).date()))
    print("T_min is " + str(min(df["T"]).date()) + "\t" + "T_max is " + str(max(df["T"]).date()) + "\n")
    
    del df
del i

index : AECO NIT MONTHLY (7A) US$
t_min is 2020-03-01	t_max is 2020-05-01
T_min is 2020-03-01	T_max is 2020-05-01

index : AECO NIT MONTHLY (7A) US$ FWD
t_min is 2020-03-02	t_max is 2020-05-29
T_min is 2020-04-01	T_max is 2022-12-01

index : NYMEX Natural Gas
t_min is 2020-03-02	t_max is 2020-05-29
T_min is 2020-04-01	T_max is 2022-12-01

index : WTI NYMEX LIGHT SWEET
t_min is 2020-03-02	t_max is 2020-05-29
T_min is 2020-03-01	T_max is 2022-12-01



In [12]:
df = prices_df[prices_df.t > prices_df["T"]]
df.head()

,index,t,T,F
4070,WTI NYMEX LIGHT SWEET,2020-03-02,2020-03-01,53.779
4104,WTI NYMEX LIGHT SWEET,2020-03-03,2020-03-01,53.779
4138,WTI NYMEX LIGHT SWEET,2020-03-04,2020-03-01,53.779
4172,WTI NYMEX LIGHT SWEET,2020-03-05,2020-03-01,53.779
4206,WTI NYMEX LIGHT SWEET,2020-03-06,2020-03-01,53.779


In [13]:
df["index"].value_counts()

WTI NYMEX LIGHT SWEET    61
Name: index, dtype: int64

In [14]:
df.nunique()

index     1
t        61
T         3
F         3
dtype: int64

In [15]:
del df

In [16]:
AECO_futures_df = grouped_by_p_i.get_group(p_i[1])
AECO_futures_df.head()

,index,t,T,F
3,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-04-01,-0.514
4,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-05-01,-0.616
5,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-06-01,-0.701
6,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-07-01,-0.706
7,AECO NIT MONTHLY (7A) US$ FWD,2020-03-02,2020-08-01,-0.754


In [17]:
AECO_futures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2049 entries, 3 to 2051
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   2049 non-null   object        
 1   t       2049 non-null   datetime64[ns]
 2   T       2049 non-null   datetime64[ns]
 3   F       2049 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 80.0+ KB


In [18]:
AECO_futures_df.to_csv(r"./data/wrangled_data/AECO_futures.csv")

In [19]:
NG_futures_df = grouped_by_p_i.get_group(p_i[2])
NG_futures_df.head()

,index,t,T,F
2052,NYMEX Natural Gas,2020-03-02,2020-04-01,1.755
2053,NYMEX Natural Gas,2020-03-02,2020-05-01,1.796
2054,NYMEX Natural Gas,2020-03-02,2020-06-01,1.857
2055,NYMEX Natural Gas,2020-03-02,2020-07-01,1.934
2056,NYMEX Natural Gas,2020-03-02,2020-08-01,1.968


In [20]:
NG_futures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2018 entries, 2052 to 4069
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   2018 non-null   object        
 1   t       2018 non-null   datetime64[ns]
 2   T       2018 non-null   datetime64[ns]
 3   F       2018 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 78.8+ KB


In [21]:
NG_futures_df.to_csv(r"./data/wrangled_data/NG_futures.csv")

In [22]:
WTI_futures_df = grouped_by_p_i.get_group(p_i[3])
WTI_futures_df = WTI_futures_df[WTI_futures_df.t < WTI_futures_df["T"]]
WTI_futures_df.head()

,index,t,T,F
4071,WTI NYMEX LIGHT SWEET,2020-03-02,2020-04-01,46.749
4072,WTI NYMEX LIGHT SWEET,2020-03-02,2020-05-01,46.919
4073,WTI NYMEX LIGHT SWEET,2020-03-02,2020-06-01,47.049
4074,WTI NYMEX LIGHT SWEET,2020-03-02,2020-07-01,47.139
4075,WTI NYMEX LIGHT SWEET,2020-03-02,2020-08-01,47.199


In [23]:
WTI_futures_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 4071 to 6129
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   1997 non-null   object        
 1   t       1997 non-null   datetime64[ns]
 2   T       1997 non-null   datetime64[ns]
 3   F       1997 non-null   float64       
dtypes: datetime64[ns](2), float64(1), object(1)
memory usage: 78.0+ KB


In [24]:
WTI_futures_df.to_csv(r"./data/wrangled_data/WTI_futures.csv")

In [25]:
NG_IV_20_df = pd.read_csv(orig_data_dir+"NG_ImpliedVols2020.csv")
NG_IV_20_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_NYMEX Natural Gas,2020-03-09,2.65,2020-11-01,0.336
1,V_NYMEX Natural Gas,2020-03-09,2.70,2020-11-01,0.336
2,V_NYMEX Natural Gas,2020-03-09,2.75,2020-11-01,0.336
3,V_NYMEX Natural Gas,2020-03-09,2.80,2020-11-01,0.336
4,V_NYMEX Natural Gas,2020-03-09,2.85,2020-11-01,0.337


In [26]:
NG_IV_20_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33212 entries, 0 to 33211
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  33212 non-null  object 
 1   volatilitydate   33212 non-null  object 
 2   strikeprice      33212 non-null  float64
 3   begtime          33212 non-null  object 
 4   volatility       33212 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


In [27]:
NG_IV_21_df = pd.read_csv(orig_data_dir+"NG_ImpliedVols2021.csv")
NG_IV_21_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_NYMEX Natural Gas,2020-03-10,1.90,2021-11-01,0.205
1,V_NYMEX Natural Gas,2020-03-10,1.95,2021-11-01,0.204
2,V_NYMEX Natural Gas,2020-03-10,2.00,2021-11-01,0.203
3,V_NYMEX Natural Gas,2020-03-10,2.05,2021-11-01,0.201
4,V_NYMEX Natural Gas,2020-03-10,2.10,2021-11-01,0.200


In [28]:
NG_IV_21_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59166 entries, 0 to 59165
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  59166 non-null  object 
 1   volatilitydate   59166 non-null  object 
 2   strikeprice      59166 non-null  float64
 3   begtime          59166 non-null  object 
 4   volatility       59166 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.3+ MB


In [29]:
NG_IV_22_df = pd.read_csv(orig_data_dir+"NG_ImpliedVols2022.csv")
NG_IV_22_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_NYMEX Natural Gas,2020-03-10,1.13,2022-01-01,0.175
1,V_NYMEX Natural Gas,2020-03-10,1.18,2022-01-01,0.179
2,V_NYMEX Natural Gas,2020-03-10,1.23,2022-01-01,0.182
3,V_NYMEX Natural Gas,2020-03-10,1.28,2022-01-01,0.185
4,V_NYMEX Natural Gas,2020-03-10,1.33,2022-01-01,0.188


In [30]:
NG_IV_22_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60480 entries, 0 to 60479
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  60480 non-null  object 
 1   volatilitydate   60480 non-null  object 
 2   strikeprice      60480 non-null  float64
 3   begtime          60480 non-null  object 
 4   volatility       60480 non-null  float64
dtypes: float64(2), object(3)
memory usage: 2.3+ MB


In [31]:
frames = [NG_IV_20_df, NG_IV_21_df, NG_IV_22_df]
NG_IV_df = pd.concat(frames)

del frames

In [32]:
NG_IV_df.rename(columns={"volatilityindex" : "index", "volatilitydate" : "t", "strikeprice" : "K",
                         "begtime" : "T", "volatility" : "sigma"}, inplace=True)
NG_IV_df = NG_IV_df[["index", "t", "T", "K", "sigma"]]
NG_IV_df["index"] = "NYMEX Natural Gas"
NG_IV_df.head()

,index,t,T,K,sigma
0,NYMEX Natural Gas,2020-03-09,2020-11-01,2.65,0.336
1,NYMEX Natural Gas,2020-03-09,2020-11-01,2.70,0.336
2,NYMEX Natural Gas,2020-03-09,2020-11-01,2.75,0.336
3,NYMEX Natural Gas,2020-03-09,2020-11-01,2.80,0.336
4,NYMEX Natural Gas,2020-03-09,2020-11-01,2.85,0.337


In [33]:
NG_IV_df["t"], NG_IV_df["T"] = pd.to_datetime(NG_IV_df["t"]), pd.to_datetime(NG_IV_df["T"])
NG_IV_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 152858 entries, 0 to 60479
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   index   152858 non-null  object        
 1   t       152858 non-null  datetime64[ns]
 2   T       152858 non-null  datetime64[ns]
 3   K       152858 non-null  float64       
 4   sigma   152858 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 7.0+ MB


In [34]:
for i in ["t", "T"]:
    print(i + "_min is " + str(min(NG_IV_df[i]).date()) + "\t" + i + "_max is " + str(max(NG_IV_df[i]).date())
          + "\n")
del i

t_min is 2020-03-02	t_max is 2020-05-29

T_min is 2020-04-01	T_max is 2022-12-01



In [35]:
WTI_IV_20_df = pd.read_csv(orig_data_dir+"WTI_ImpliedVols2020.csv")
WTI_IV_20_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_WTI NYMEX LIGHT SWEET,2020-03-06,25.0,2020-11-01,0.475
1,V_WTI NYMEX LIGHT SWEET,2020-03-06,25.5,2020-11-01,0.471
2,V_WTI NYMEX LIGHT SWEET,2020-03-06,26.0,2020-11-01,0.467
3,V_WTI NYMEX LIGHT SWEET,2020-03-06,26.5,2020-11-01,0.463
4,V_WTI NYMEX LIGHT SWEET,2020-03-06,27.0,2020-11-01,0.458


In [36]:
WTI_IV_20_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32968 entries, 0 to 32967
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  32968 non-null  object 
 1   volatilitydate   32968 non-null  object 
 2   strikeprice      32968 non-null  float64
 3   begtime          32968 non-null  object 
 4   volatility       32968 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


In [37]:
WTI_IV_21_df = pd.read_csv(orig_data_dir+"WTI_ImpliedVols2021.csv")
WTI_IV_21_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_WTI NYMEX LIGHT SWEET,2020-03-06,27.0,2021-01-01,0.423
1,V_WTI NYMEX LIGHT SWEET,2020-03-06,27.5,2021-01-01,0.419
2,V_WTI NYMEX LIGHT SWEET,2020-03-06,28.0,2021-01-01,0.415
3,V_WTI NYMEX LIGHT SWEET,2020-03-06,28.5,2021-01-01,0.411
4,V_WTI NYMEX LIGHT SWEET,2020-03-06,29.0,2021-01-01,0.407


In [38]:
WTI_IV_21_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35134 entries, 0 to 35133
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  35134 non-null  object 
 1   volatilitydate   35134 non-null  object 
 2   strikeprice      35134 non-null  float64
 3   begtime          35134 non-null  object 
 4   volatility       35134 non-null  float64
dtypes: float64(2), object(3)
memory usage: 1.3+ MB


In [39]:
WTI_IV_22_df = pd.read_csv(orig_data_dir+"WTI_ImpliedVols2022.csv")
WTI_IV_22_df.head()

,volatilityindex,volatilitydate,strikeprice,begtime,volatility
0,V_WTI NYMEX LIGHT SWEET,2020-03-06,31.5,2022-06-01,0.265
1,V_WTI NYMEX LIGHT SWEET,2020-03-06,32.0,2022-06-01,0.265
2,V_WTI NYMEX LIGHT SWEET,2020-03-06,32.5,2022-06-01,0.265
3,V_WTI NYMEX LIGHT SWEET,2020-03-06,33.0,2022-06-01,0.265
4,V_WTI NYMEX LIGHT SWEET,2020-03-06,33.5,2022-06-01,0.264


In [40]:
WTI_IV_22_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9639 entries, 0 to 9638
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   volatilityindex  9639 non-null   object 
 1   volatilitydate   9639 non-null   object 
 2   strikeprice      9639 non-null   float64
 3   begtime          9639 non-null   object 
 4   volatility       9639 non-null   float64
dtypes: float64(2), object(3)
memory usage: 376.6+ KB


In [41]:
frames = [WTI_IV_20_df, WTI_IV_21_df, WTI_IV_22_df]
WTI_IV_df = pd.concat(frames)

del frames

In [42]:
WTI_IV_df.rename(columns={"volatilityindex" : "index", "volatilitydate" : "t", "strikeprice" : "K",
                         "begtime" : "T", "volatility" : "sigma"}, inplace=True)
WTI_IV_df = WTI_IV_df[["index", "t", "T", "K", "sigma"]]
WTI_IV_df["index"] = "WTI NYMEX LIGHT SWEET"
WTI_IV_df.head()

,index,t,T,K,sigma
0,WTI NYMEX LIGHT SWEET,2020-03-06,2020-11-01,25.0,0.475
1,WTI NYMEX LIGHT SWEET,2020-03-06,2020-11-01,25.5,0.471
2,WTI NYMEX LIGHT SWEET,2020-03-06,2020-11-01,26.0,0.467
3,WTI NYMEX LIGHT SWEET,2020-03-06,2020-11-01,26.5,0.463
4,WTI NYMEX LIGHT SWEET,2020-03-06,2020-11-01,27.0,0.458


In [43]:
WTI_IV_df["t"], WTI_IV_df["T"] = pd.to_datetime(WTI_IV_df["t"]), pd.to_datetime(WTI_IV_df["T"])
WTI_IV_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77741 entries, 0 to 9638
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   index   77741 non-null  object        
 1   t       77741 non-null  datetime64[ns]
 2   T       77741 non-null  datetime64[ns]
 3   K       77741 non-null  float64       
 4   sigma   77741 non-null  float64       
dtypes: datetime64[ns](2), float64(2), object(1)
memory usage: 3.6+ MB


In [44]:
for i in ["t", "T"]:
    print(i + "_min is " + str(min(WTI_IV_df[i]).date()) + "\t" + i + "_max is " + str(max(WTI_IV_df[i]).date())
          + "\n")
del i

t_min is 2020-03-02	t_max is 2020-05-29

T_min is 2020-04-01	T_max is 2022-12-01



In [45]:
IR_df = pd.read_csv(orig_data_dir+"InterestRates.csv")
IR_df.head()

,pricedate,maturity,bidrate
0,2020-03-02,100M,1.463333
1,2020-03-02,101M,1.466667
2,2020-03-02,102M,1.470001
3,2020-03-02,103M,1.478335
4,2020-03-02,104M,1.486669


In [46]:
IR_df.pricedate = pd.to_datetime(IR_df.pricedate)
IR_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7085 entries, 0 to 7084
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   pricedate  7085 non-null   datetime64[ns]
 1   maturity   7085 non-null   object        
 2   bidrate    7085 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 166.2+ KB


In [47]:
t = IR_df.pricedate.to_list()
ttm = [to_offset(i) for i in IR_df.maturity]
T = [t[i] + ttm[i] + to_offset("1D") for i in range(len(ttm))]
IR_df.maturity = T

In [48]:
IR_df.head()

,pricedate,maturity,bidrate
0,2020-03-02,2028-07-01,1.463333
1,2020-03-02,2028-08-01,1.466667
2,2020-03-02,2028-09-01,1.470001
3,2020-03-02,2028-10-01,1.478335
4,2020-03-02,2028-11-01,1.486669


In [49]:
IR_df.rename(columns={"pricedate" : "t", "maturity" : "T", "bidrate": "r"}, inplace=True)
IR_df.head()

,t,T,r
0,2020-03-02,2028-07-01,1.463333
1,2020-03-02,2028-08-01,1.466667
2,2020-03-02,2028-09-01,1.470001
3,2020-03-02,2028-10-01,1.478335
4,2020-03-02,2028-11-01,1.486669


In [50]:
for i in ["t", "T"]:
    print(i + "_min is " + str(min(IR_df[i]).date()) + "\t" + i + "_max is " + str(max(IR_df[i]).date())
          + "\n")
del i

t_min is 2020-03-02	t_max is 2020-05-29

T_min is 2020-04-01	T_max is 2029-06-01



In [51]:
NG_IV_df.set_index(["t", "T"], inplace=True)
WTI_IV_df.set_index(["t", "T"], inplace=True)
IR_df.set_index(["t", "T"], inplace=True)

In [52]:
NG_option_df = NG_IV_df.merge(IR_df, how="inner", left_index=True, right_index=True)
NG_option_df.reset_index()
NG_option_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 152857 entries, (Timestamp('2020-03-02 00:00:00'), Timestamp('2020-04-01 00:00:00')) to (Timestamp('2020-05-29 00:00:00'), Timestamp('2022-12-01 00:00:00'))
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   index   152857 non-null  object 
 1   K       152857 non-null  float64
 2   sigma   152857 non-null  float64
 3   r       152857 non-null  float64
dtypes: float64(3), object(1)
memory usage: 5.0+ MB


In [53]:
NG_option_df.to_csv(r"./data/wrangled_data/NG_option.csv")

In [54]:
WTI_option_df = WTI_IV_df.merge(IR_df, how="inner", left_index=True, right_index=True)
WTI_option_df.reset_index()
WTI_option_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 79354 entries, (Timestamp('2020-03-02 00:00:00'), Timestamp('2020-04-01 00:00:00')) to (Timestamp('2020-05-29 00:00:00'), Timestamp('2022-12-01 00:00:00'))
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   index   79354 non-null  object 
 1   K       79354 non-null  float64
 2   sigma   79354 non-null  float64
 3   r       79354 non-null  float64
dtypes: float64(3), object(1)
memory usage: 2.6+ MB


In [55]:
NG_option_df.to_csv(r"./data/wrangled_data/WTI_option.csv")

In [56]:
print(*os.listdir("./data/wrangled_data/"), sep="\n")

WTI_futures.csv
NG_option.csv
WTI_option.csv
AECO_futures.csv
NG_futures.csv
